In [1]:
# Import the necessary dependencies.
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.stats as sts
import plotly.express as plt
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

## The Data

These data sets are from fitbit. They contain 10K+ rows of data regarding the activities and sleep activites of about 20+ people. To clean the data, we took the daily averages of their sleep, steps, calories, and distance walked/ran. Using that condensed data, we created scatterplots analyzing the amount of sleep, amount of minutes sedentary, and daily steps. We also analyzed the time in bed, time asleep, and steps. 

Questions: 
- Does being more active (i.e. daily steps, active minutes) reduce the amount of time it takes to fall asleep?
- Does sedentary time affect the amount of sleep?

**Create a null hypothesis, an alternative hypothesis, and choose a significance level. Use this cell to document your decisions.**

Question: Does being more sedentary affect the amount of sleep you get?

Hypothesis: If you are more sedentary throughout the day, you will sleep less.

Null Hypothesis: Being sedentary more than 900 minutes per day will not decrease the amount of sleep.

Alternative Hypothesis: Being sedentary more than 900 minutes per day will decrease the amount of sleep. 

Significance Level:  Our p values for sedentaryMinutes and TotalSteps are less 0.05, so we reject the null hypothesis.


### Other Notes

- The less sedentary, the more sleep people got. 
- Total steps walked has a 0.2 correlation with total minutes asleep.
- Based on the linear regeression model, we reject the null hypothesis.

In [2]:
# Get actvity DF for all users
sedentary_df = pd.read_csv('fitbit/dailyActivity_merged.csv')
sedentary_df = sedentary_df.groupby('Id').mean()
sedentary_df.head()

,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
Id,,,,,,,,,,,,,
1503960366,12116.741935,7.809677,7.809677,0.0,2.858387,0.794194,4.152903,0.000000,38.709677,19.161290,219.935484,848.161290,1816.419355
1624580081,5743.903226,3.914839,3.914839,0.0,0.939355,0.360645,2.606774,0.006129,8.677419,5.806452,153.483871,1257.741935,1483.354839
1644430081,7282.966667,5.295333,5.295333,0.0,0.730000,0.951000,3.609000,0.004000,9.566667,21.366667,178.466667,1161.866667,2811.300000
1844505072,2580.064516,1.706129,1.706129,0.0,0.008387,0.049032,1.647419,0.000000,0.129032,1.290323,115.451613,1206.612903,1573.483871
1927972279,916.129032,0.634516,0.634516,0.0,0.095806,0.031290,0.507097,0.000000,1.322581,0.774194,38.580645,1317.419355,2172.806452


In [3]:
sleep_df = pd.read_csv('fitbit/sleepDay_merged.csv')
sleep_df = sleep_df.groupby('Id').mean()
sleep_df['TimeToFallAsleep'] = sleep_df['TotalTimeInBed'] - sleep_df['TotalMinutesAsleep']
sleep_df.head()


,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,TimeToFallAsleep
Id,,,,
1503960366,1.08,360.280000,383.200000,22.920000
1644430081,1.00,294.000000,346.000000,52.000000
1844505072,1.00,652.000000,961.000000,309.000000
1927972279,1.60,417.000000,437.800000,20.800000
2026352035,1.00,506.178571,537.642857,31.464286


In [4]:
combined_df = sleep_df.merge(sedentary_df, on='Id')
combined_df.drop_duplicates()
combined_df.head()

,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,TimeToFallAsleep,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
Id,,,,,,,,,,,,,,,,,
1503960366,1.08,360.280000,383.200000,22.920000,12116.741935,7.809677,7.809677,0.0,2.858387,0.794194,4.152903,0.000,38.709677,19.161290,219.935484,848.161290,1816.419355
1644430081,1.00,294.000000,346.000000,52.000000,7282.966667,5.295333,5.295333,0.0,0.730000,0.951000,3.609000,0.004,9.566667,21.366667,178.466667,1161.866667,2811.300000
1844505072,1.00,652.000000,961.000000,309.000000,2580.064516,1.706129,1.706129,0.0,0.008387,0.049032,1.647419,0.000,0.129032,1.290323,115.451613,1206.612903,1573.483871
1927972279,1.60,417.000000,437.800000,20.800000,916.129032,0.634516,0.634516,0.0,0.095806,0.031290,0.507097,0.000,1.322581,0.774194,38.580645,1317.419355,2172.806452
2026352035,1.00,506.178571,537.642857,31.464286,5566.870968,3.454839,3.454839,0.0,0.006129,0.011290,3.436129,0.000,0.096774,0.258065,256.645161,689.419355,1540.645161


In [5]:
# Note: Average data
px.scatter(combined_df,x='TotalMinutesAsleep', y='SedentaryMinutes', color='TotalSteps')

In [6]:
# Note: Average data
px.scatter(combined_df,x='TotalMinutesAsleep', y='TotalTimeInBed', color='SedentaryMinutes')

In [1]:
# Scatter plot to see how the time it takes to fall asleep is related to being slightly active
px.scatter(combined_df,x='LightlyActiveMinutes', y='TimeToFallAsleep')

NameError: name 'px' is not defined

In [8]:
combined_df.corr()

,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,TimeToFallAsleep,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
TotalSleepRecords,1.000000,0.180406,0.118174,-0.063441,-0.182006,-0.158641,-0.155259,-0.107772,-0.159170,-0.087964,-0.076454,-0.151639,-0.154466,-0.141955,-0.121234,-0.053196,0.034278
TotalMinutesAsleep,0.180406,1.000000,0.940230,0.411569,-0.218571,-0.207983,-0.193880,-0.371701,-0.156875,-0.169274,-0.139836,-0.042635,-0.064302,-0.144973,-0.202731,-0.376279,-0.040646
TotalTimeInBed,0.118174,0.940230,1.000000,0.697332,-0.240679,-0.236997,-0.224672,-0.342326,-0.194964,-0.066477,-0.183736,-0.063349,-0.122541,-0.057641,-0.230326,-0.270743,-0.167031
TimeToFallAsleep,-0.063441,0.411569,0.697332,1.000000,-0.184087,-0.196517,-0.193217,-0.133822,-0.191594,0.178366,-0.197408,-0.079803,-0.192612,0.150867,-0.189718,0.067387,-0.361469
TotalSteps,-0.182006,-0.218571,-0.240679,-0.184087,1.000000,0.985777,0.984553,0.256617,0.729549,0.581238,0.690334,-0.059438,0.632106,0.537021,0.563201,-0.506390,0.321572
TotalDistance,-0.158641,-0.207983,-0.236997,-0.196517,0.985777,1.000000,0.999460,0.267496,0.763837,0.551887,0.682294,-0.072000,0.669048,0.511986,0.514461,-0.462965,0.443382
TrackerDistance,-0.155259,-0.193880,-0.224672,-0.193217,0.984553,0.999460,1.000000,0.239702,0.765356,0.553787,0.678045,-0.076275,0.671056,0.514568,0.508139,-0.469016,0.444022
LoggedActivitiesDistance,-0.107772,-0.371701,-0.342326,-0.133822,0.256617,0.267496,0.239702,1.000000,0.169885,0.040930,0.288349,0.060285,0.227570,-0.000241,0.217905,-0.014431,0.275310
VeryActiveDistance,-0.159170,-0.156875,-0.194964,-0.191594,0.729549,0.763837,0.765356,0.169885,1.000000,0.183003,0.091617,-0.091063,0.877630,0.235211,-0.013398,-0.058888,0.455649
ModeratelyActiveDistance,-0.087964,-0.169274,-0.066477,0.178366,0.581238,0.551887,0.553787,0.040930,0.183003,1.000000,0.402310,0.047208,0.173267,0.955380,0.241151,-0.425531,0.032014


In [10]:
import statsmodels.api as sm
# Select your independent X terms, and your dependent y term. 
X = combined_df[['SedentaryMinutes', 'TotalSteps']]
y = combined_df['TotalMinutesAsleep']
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
# Print the linear regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     TotalMinutesAsleep   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     6.079
Date:                Sat, 23 Apr 2022   Prob (F-statistic):            0.00826
Time:                        13:54:33   Log-Likelihood:                -146.10
No. Observations:                  24   AIC:                             298.2
Df Residuals:                      21   BIC:                             301.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              896.9624    150.836  